# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [118]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [119]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [120]:
# confirm that our new database was created
db_list = mongo.list_database_names()
print(db_list)

['admin', 'config', 'local', 'uk_food']


In [121]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [122]:
# Review the collections in our new database
collections_list = db.list_collection_names()
print(collections_list)


['establishments']


In [123]:
# review the collections in our new database
collections_list = db.list_collection_names()
print(collections_list)


['establishments']


In [124]:
# review a document in the establishments collection
establishment_sample = db.establishments.find_one()
pprint(establishment_sample)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('664c0cb3e65a6aaf123dc556'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extrac

In [125]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [126]:
# Create a dictionary for the new restaurant data
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant",
    "BusinessTypeID": 9999,
    "AddressLine1": "Greenwich Street",
    "AddressLine2": "Greenwich",
    "PostCode": "SE10 9JN",
    "RatingValue": "Awaiting Inspection",
    "LocalAuthorityCode": "184",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.greenwich.gov.uk/",
    "LocalAuthorityEmailAddress": "foodsafety@greenwich.gov.uk",
    "RatingDate": None,
    "SchemeType": "FHRS",
    "NewRatingPending": True
}


In [127]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)
print("Inserted restaurant with id:", result.inserted_id)



Inserted restaurant with id: 664c539a06c80d246e036884


In [128]:
# Check that the new restaurant was inserted
new_restaurant_data = establishments.find_one({'BusinessName': 'Penang Flavours'})
pprint(new_restaurant_data)

{'AddressLine1': 'Greenwich Street',
 'AddressLine2': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant',
 'BusinessTypeID': 1,
 'LocalAuthorityCode': '184',
 'LocalAuthorityEmailAddress': 'foodsafety@greenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.greenwich.gov.uk/',
 'NewRatingPending': True,
 'PostCode': 'SE10 9JN',
 'RatingDate': None,
 'RatingValue': None,
 'SchemeType': 'FHRS',
 '_id': ObjectId('664c186a06c80d246e03687d'),
 'geocode': {'latitude': 'Error', 'longitude': 'Error'}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [129]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
restaurant_type_info = establishments.find_one(
    {'BusinessType': 'Restaurant/Cafe/Canteen'},
    {'_id': 0, 'BusinessTypeID': 1, 'BusinessType': 1}
)
pprint(restaurant_type_info)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [130]:
# Update the new restaurant with the correct BusinessTypeID
update_result = establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set': {'BusinessTypeID': 1}}
)
print(f"Documents modified: {update_result.modified_count}")

Documents modified: 0


In [131]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({'BusinessName': 'Penang Flavours'})
pprint(updated_restaurant)



{'AddressLine1': 'Greenwich Street',
 'AddressLine2': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant',
 'BusinessTypeID': 1,
 'LocalAuthorityCode': '184',
 'LocalAuthorityEmailAddress': 'foodsafety@greenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.greenwich.gov.uk/',
 'NewRatingPending': True,
 'PostCode': 'SE10 9JN',
 'RatingDate': None,
 'RatingValue': None,
 'SchemeType': 'FHRS',
 '_id': ObjectId('664c186a06c80d246e03687d'),
 'geocode': {'latitude': 'Error', 'longitude': 'Error'}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [132]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print("Number of establishments in Dover:", dover_count)

Number of establishments in Dover: 0


In [133]:
# Delete all documents with LocalAuthorityName as "Dover"
delete_result = establishments.delete_many({'LocalAuthorityName': 'Dover'})
print("Number of documents deleted:", delete_result.deleted_count)


Number of documents deleted: 0


In [134]:
# Check if any remaining documents include Dover
remaining_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})
print("Remaining documents with Dover:", remaining_count)


Remaining documents with Dover: 0


In [135]:
# Check that other documents remain with 'find_one'
sample_document = establishments.find_one()
print("Sample document from the remaining collection:", sample_document)



Sample document from the remaining collection: {'_id': ObjectId('664c0cb3e65a6aaf123dc556'), 'FHRSID': 647177, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000041489', 'BusinessName': 'Wear Bay Bowls Club', 'BusinessType': 'Pub/bar/nightclub', 'BusinessTypeID': 7843, 'AddressLine1': 'Wear Bay Road', 'AddressLine2': 'Folkestone', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT19 6PY', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2014-03-31T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10}, 'SchemeType': 'FHRS', 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}, 'RightToReply': '', 'Distance': 4591.821311183521, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T0

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [136]:
# Change the data type from String to Decimal for longitude and latitude
from bson.decimal128 import Decimal128

result = establishments.update_many(
    {},
    [
        {
            '$set': {
                'geocode.longitude': {
                    '$convert': {
                        'input': '$geocode.longitude',
                        'to': 'double',
                        'onError': 'Error',
                        'onNull': 'Error'
                    }
                },
                'geocode.latitude': {
                    '$convert': {
                        'input': '$geocode.latitude',
                        'to': 'double',
                        'onError': 'Error',
                        'onNull': 'Error'
                    }
                }
            }
        }
    ]
)

print("Documents modified:", result.modified_count)

Documents modified: 1


Use `update_many` to convert `RatingValue` to integer numbers.

In [137]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [138]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [139]:
# Check that the coordinates and rating value are now numbers
count_correct_types = establishments.count_documents({
    "geocode.longitude": {"$type": "double"},
    "geocode.latitude": {"$type": "double"},
    "RatingValue": {"$type": "int"} 
})

print(f"Number of documents with numeric longitude, latitude, and integer RatingValue: {count_correct_types}")

Number of documents with numeric longitude, latitude, and integer RatingValue: 34694
